# Data Science Capstone Project: Part 2
Autor: Sébastien SIME
___

## Table of content:

1. Read Toronto geospatial Data (Using the provide csv file)
2. Include geospatial data into Toronto_neighbourhood data frame (previouslyy created in part 1)

## 1. Read Toronto geospatial Data:

### Using the csv file

In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Option 1: read geospatial using the provided csv file (for info geocode didn't work at all)
geospatial = pd.read_csv("Geospatial_Coordinates.csv")
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
geospatial.shape

(103, 3)

In [4]:
geospatial.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [9]:
# Check for duplicated values
geospatial.duplicated().sum()

0

## 2. Include geospatial coordinates into the Toronto_neighbourhood data frame:

In [11]:
# Read the previoulsly created toronto_neighborhood data frame (Part 1 of Capstone project)
toronto_neighbourhood = pd.read_csv('Toronto_neighbourhood.csv')
toronto_neighbourhood.head(12)

,Unnamed: 0,PostalCode,Borough,Neighbourhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,6,M1B,Scarborough,"Malvern, Rouge"
7,7,M3B,North York,Don Mills
8,8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
# Let's drop the Unammed column:
toronto_neighbourhood.drop(columns = 'Unnamed: 0', inplace = True)
toronto_neighbourhood.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
toronto_neighbourhood.dtypes

PostalCode       object
Borough          object
Neighbourhood    object
dtype: object

In [14]:
toronto_neighbourhood.shape

(103, 3)

The same dimension that the geospatial data frame

In [15]:
# Some postal code check:
postalList = ['M3A', 'M7A', 'M8X']
geospatial.loc[geospatial['Postal Code'].isin(postalList), :]

,Postal Code,Latitude,Longitude
25,M3A,43.753259,-79.329656
85,M7A,43.662301,-79.389494
90,M8X,43.653654,-79.506944


In [16]:
# join Operation between toronto_neighbourhood and geospatial
toronto_neighbourhood_merged = toronto_neighbourhood.join(geospatial.set_index('Postal Code'), on="PostalCode")
toronto_neighbourhood_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [21]:
# Check for duplicated
toronto_neighbourhood_merged['Neighbourhood'].duplicated().sum()

4

In [23]:
dup = toronto_neighbourhood_merged['Neighbourhood'].duplicated()
toronto_neighbourhood_merged.loc[dup, :]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
13,M3C,North York,Don Mills,43.725900,-79.340923
46,M3L,North York,Downsview,43.739015,-79.506944
53,M3M,North York,Downsview,43.728496,-79.495697
60,M3N,North York,Downsview,43.761631,-79.520999


We can see that thera are some duplicated rows but with distinct Postal Code. Let's check on https://canadapostcode.net/location/CA-ON/Toronto for precision regarding the postal code. I found that:
* M3C: Don Mills South
* M3L: Downsview West
* M3M: Downsview Central
* M3N: North York (Jane and Finch)

In [26]:
# Lets update the values
ind = [13, 46, 53, 60]
name = ['Don Mills South', 'Downsview West', 'Downsview Central', 'North York (Jane and Finch)']
#
for i, nam in zip (ind, name):
    toronto_neighbourhood_merged.loc[i, 'Neighbourhood'] = nam
#
toronto_neighbourhood_merged.iloc[ind, :]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
13,M3C,North York,Don Mills South,43.725900,-79.340923
46,M3L,North York,Downsview West,43.739015,-79.506944
53,M3M,North York,Downsview Central,43.728496,-79.495697
60,M3N,North York,North York (Jane and Finch),43.761631,-79.520999


In [27]:
# Let's check for duplicates AGAIN!
toronto_neighbourhood_merged['Neighbourhood'].duplicated().sum()

0

In [31]:
# Display the dimensions
#toronto_neighbourhood_merged.shape
toronto_neighbourhood_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [30]:
# Export to csv
toronto_neighbourhood_merged.to_csv('toronto_neighbourhood_nd_spatial.csv')
print('part 2 done!')

part 2 done!


___